In [10]:
from google.colab import drive
import sys
import os

drive.mount('/content/drive')

# Personalized path to your project directory
path = '/content/drive/MyDrive/cod/LEA3_RecSys' # Replace with your actual path

# Add the path to sys.path
sys.path.append(path)
os.chdir(path)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import numpy as np
import pandas as pd
import sqlite3 as sql
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact ## para análisis interactivo
from sklearn import neighbors ### basado en contenido un solo producto consumido
import joblib




In [12]:

#############################################
#### conectar_base_de_Datos#################
############################################

conn=sql.connect('data/db_books2')
cur=conn.cursor()

cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())


[('books',), ('book_ratings',), ('users',), ('usuarios_sel',), ('books_sel',), ('ratings_final',), ('users_final',), ('books_final',), ('full_ratings',)]


In [13]:
#######################################################################
#### 3 Sistema de recomendación basado en contenido KNN #################
#### Con base en todo lo visto por el usuario #######################
#######################################################################



##### cargar data frame escalado y con dummies ###
books_dum2= joblib.load('salidas/books_dum2.joblib')

### carga data frame normal que tiene nombres de libros
books=pd.read_sql('select * from books_final', conn )
#books['year_pub']=books.year_pub.astype('int')



#### seleccionar usuario para recomendaciones ####
usuarios=pd.read_sql('select distinct (user_id) as user_id from ratings_final',conn)

user_id=31226 ### para ejemplo manual


def recomendar(user_id=list(usuarios['user_id'].value_counts().index)):

    ###seleccionar solo los ratings del usuario seleccionado
    ratings=pd.read_sql('select *from ratings_final where user_id=:user',conn, params={'user':user_id})

    ###convertir ratings del usuario a array
    l_books_r=ratings['isbn'].to_numpy()

    ###agregar la columna de isbn y titulo del libro a dummie para filtrar y mostrar nombre
    books_dum2[['isbn','book_title']]=books[['isbn','book_title']]

    ### filtrar libros calificados por el usuario
    books_r=books_dum2[books_dum2['isbn'].isin(l_books_r)]

    ## eliminar columna nombre e isbn
    books_r=books_r.drop(columns=['isbn','book_title'])
    books_r["indice"]=1 ### para usar group by y que quede en formato pandas tabla de centroide
    ##centroide o perfil del usuario
    centroide=books_r.groupby("indice").mean()


    ### filtrar libros no leídos
    books_nr=books_dum2[~books_dum2['isbn'].isin(l_books_r)]
    ## eliminbar nombre e isbn
    books_nr=books_nr.drop(columns=['isbn','book_title'])

    ### entrenar modelo
    model=neighbors.NearestNeighbors(n_neighbors=11, metric='cosine')
    model.fit(books_nr)
    dist, idlist = model.kneighbors(centroide)

    ids=idlist[0] ### queda en un array anidado, para sacarlo
    recomend_b=books.loc[ids][['book_title','isbn']]
    leidos=books[books['isbn'].isin(l_books_r)][['book_title','isbn']]

    return recomend_b


recomendar(52853)


print(interact(recomendar))

interactive(children=(Dropdown(description='user_id', options=(79961, 31226, 52853, 167471, 119517, 8066, 2820…

<function recomendar at 0x7a2a8f2a2e80>


In [14]:
#instalar lightfm para sistema de recomendación de filtro colaborativo


!pip install lightfm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp311-cp311-linux_x86_64.whl size=831166 sha256=cac9b3d6beb1cf8a0b2422629236d643ba476c52ba4dc317bb7c22bac9bec25f
  Stored in directory: /root/.cache/pip/wheels/b9/0d/8a/0729d2e6e3ca2a898ba55201f905da7db3f838a33df5b3fcdd
Successfully built lightfm


In [15]:
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import auc_score
from sklearn.model_selection import train_test_split
#from scipy.sparse import coo_matrix
#from sklearn.model_selection import GridSearchCV

In [16]:
ratings = pd.read_sql('SELECT * FROM full_ratings ', conn)

#Crear data set en formato lightfm, primero lista de usuarios unicos y items únicos
dataset_train = Dataset()
dataset_test = Dataset()

all_unique_users = ratings['user_id'].unique()
all_unique_items = ratings['isbn'].unique()

dataset_train.fit(users=all_unique_users, items=all_unique_items)
dataset_test.fit(users=all_unique_users, items=all_unique_items)



In [17]:

## verificar que el data set creado tenga los mismos registros que base original
cur.execute('SELECT count(distinct user_id), count (distinct isbn) FROM full_ratings ')
print(cur.fetchall())

num_users, num_items = dataset_train.interactions_shape()
print(num_users, num_items)




[(2427, 1377)]
2427 1377


In [18]:
#### separar en train y test
train_df, test_df = train_test_split(ratings, test_size=0.2, random_state=42)

### para trabajar con el paquete light FM, se deben crear datos que crean consecutivos para cada usuario e item

train_interactions_list = [(row['user_id'], row['isbn'], row['book_rating']) for index, row in train_df.iterrows()]

train_interactions, train_weights = dataset_train.build_interactions(train_interactions_list)

test_interactions_list = [(row['user_id'], row['isbn'], row['book_rating']) for index, row in test_df.iterrows()]
test_interactions, test_weights = dataset_test.build_interactions(test_interactions_list)


In [19]:
print(f"Shape of Interactions Matrix: {train_interactions.shape}")
print(f"Shape of Weights Matrix: {train_weights.shape}")
print(f"Shape of Interactions Matrix: {test_interactions.shape}")
print(f"Shape of Weights Matrix: {test_weights.shape}")

Shape of Interactions Matrix: (2427, 1377)
Shape of Weights Matrix: (2427, 1377)
Shape of Interactions Matrix: (2427, 1377)
Shape of Weights Matrix: (2427, 1377)


In [20]:
#Este código es de verificación para mirar los datos cómo se estructuran
train_int_array=train_interactions.toarray()
train_weights_array=train_weights.toarray()

test_int_array=test_interactions.toarray()
test_weights_array=test_weights.toarray()

print(train_int_array)
print(test_int_array)

print(train_weights_array)
print(test_weights_array)



[[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [5]:
!pip install --quiet optuna
import optuna
import time


In [ ]:
############# ajuste manual del modelo
loss_model='logistic'  ## 'logistic', 'bpr', 'warp', 'warp-kos': este último solo sirve para implicitas

model=LightFM(loss=loss_model, random_state=42)
model.fit(train_interactions, epochs=20, verbose=True, sample_weight=train_weights)

### auc por usuario
train_auc = auc_score(model, train_interactions)
test_auc = auc_score(model, test_interactions)


print('AUC: train %.2f, test %.2f.' % (train_auc.mean(), test_auc.mean()))




Epoch: 100%|██████████| 20/20 [00:00<00:00, 27.15it/s]


AUC: train 0.68, test 0.63.


In [39]:
#### para afinar hiperparámetros con optuna

def objective(trial):

    # grilla de hiperparámetros
    n_components = trial.suggest_int('no_components', 10, 100)       # Sugiere un entero para el número de componentes latentes (dimensionalidad del modelo).
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1, log=True) # Sugiere un flotante (en escala logarítmica) para la tasa de aprendizaje del optimizador.
    loss = trial.suggest_categorical('loss', ['logistic', 'bpr', 'warp']) # Sugiere un valor categórico para la función de pérdida ('logistic', 'bpr' o 'warp').
    epochs = trial.suggest_int('epochs', 10, 50)                     # Sugiere un entero para el número de épocas de entrenamiento.

    print(f"  Params: n_components={n_components}, lr={learning_rate:.4f}, loss={loss},  epochs={epochs}")

    # Modelo con parámetros sugeridos
    model = LightFM(
        no_components=n_components,
        learning_rate=learning_rate,
        loss=loss,

        random_state=42
    )


    model.fit(train_interactions,
              epochs=epochs,
              verbose=False,
              sample_weight=train_weights)

    test_auc = auc_score(model, test_interactions).mean()



    return test_auc


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10) # You can adjust the number of trials

#Resultados
print(f"Best trial:")
print(f"  Value (Test AUC): {study.best_value:.4f}")
print(f"  Params: {study.best_params}")

#Entrenar modelo con mejores hyperparametros
best_params = study.best_params


model = LightFM(
    no_components=best_params['no_components'],
    learning_rate=best_params['learning_rate'],
    loss=best_params['loss'],
    random_state=42
)


model.fit(train_interactions,
                epochs=best_params['epochs'],
                verbose=True, # Set to True to see progress
                sample_weight=train_weights)


### resultado final
train_auc = auc_score(model, train_interactions).mean()
test_auc = auc_score(model, test_interactions).mean()

print('Final Model AUC: train %.4f, test %.4f.' % (train_auc, test_auc))

[I 2025-05-07 17:35:20,130] A new study created in memory with name: no-name-270a4a7b-0270-4eb3-9743-a0f3ae087114


  Params: n_components=68, lr=0.0150, loss=bpr,  epochs=30


[I 2025-05-07 17:35:27,279] Trial 0 finished with value: 0.6034123301506042 and parameters: {'no_components': 68, 'learning_rate': 0.014997066146152848, 'loss': 'bpr', 'epochs': 30}. Best is trial 0 with value: 0.6034123301506042.


  Params: n_components=16, lr=0.0143, loss=warp,  epochs=13


[I 2025-05-07 17:35:28,188] Trial 1 finished with value: 0.622983455657959 and parameters: {'no_components': 16, 'learning_rate': 0.014268579175894507, 'loss': 'warp', 'epochs': 13}. Best is trial 1 with value: 0.622983455657959.


...

  Params: n_components=64, lr=0.0087, loss=bpr,  epochs=19


[I 2025-05-07 17:36:06,080] Trial 9 finished with value: 0.5631009340286255 and parameters: {'no_components': 64, 'learning_rate': 0.008664901891539676, 'loss': 'bpr', 'epochs': 19}. Best is trial 8 with value: 0.6571308374404907.


Best trial:
  Value (Test AUC): 0.6571
  Params: {'no_components': 36, 'learning_rate': 0.02053831201977882, 'loss': 'warp', 'epochs': 21}


Epoch: 100%|██████████| 21/21 [00:02<00:00,  9.96it/s]


Final Model AUC: train 0.7469, test 0.6571.


In [53]:
list(dataset_train.mapping()[0].items())[0:10] ### para  mapeo de primerios 10 codigos originales a codigos del modelo

[(np.int64(31226), 0),
 (np.int64(52853), 1),
 (np.int64(167471), 2),
 (np.int64(119517), 3),
 (np.int64(8066), 4),
 (np.int64(28204), 5),
 (np.int64(28619), 6),
 (np.int64(113519), 7),
 (np.int64(179733), 8),
 (np.int64(152186), 9)]

In [54]:
def recommendation(model, data, original_user_id, conn,k):

  #data=dataset_train
  #original_user_id=52853

  ## libros no leidos y códigos
  df_nr_books=pd.read_sql(f'select * from full_ratings where user_id<>{original_user_id}',conn)
  isbn_nr_books=df_nr_books['isbn'].values
  item_id_nr=[value for key, value in data.mapping()[2].items() if value not in isbn_nr_books]

  uid_index=data.mapping()[0][original_user_id] ## id usuario según modelo

  scores=model.predict(uid_index, item_id_nr)
  sorted_indices = np.argsort(-scores).tolist()

  top_items = [key for key, value in data.mapping()[2].items() if value in sorted_indices[:k]]
  recommended=df_nr_books[df_nr_books['isbn'].isin(top_items)][['isbn', 'book_title']]
  recommended.drop_duplicates(inplace=True)

  return recommended


In [55]:
recommendation(model, dataset_train,92652, conn, 10)

,isbn,book_title
36,0971880107,Wild Animus
90,0345313860,"The Vampire Lestat (Vampire Chronicles, Book II)"
216,0345351525,The Queen of the Damned (Vampire Chronicles (P...
294,155874262X,Chicken Soup for the Soul (Chicken Soup for th...
386,0451156609,The Tommyknockers
579,0316769487,The Catcher in the Rye
595,0451184963,Insomnia
596,0385504209,The Da Vinci Code
1056,0446605239,The Notebook
6890,0684814994,Christmas Box (Christmas Box Trilogy)


In [56]:
import nbformat

# Path to your notebook
input_notebook = 'd_modelos2.ipynb'
output_notebook = 'd_modelos22.ipynb'

# Load the notebook
with open(input_notebook, 'r') as f:
    notebook = nbformat.read(f, as_version=4)

# Check if the notebook has 'metadata.widgets' and remove it
if 'widgets' in notebook.metadata:
    del notebook.metadata['widgets']  # Removes widgets metadata completely

# Alternatively, if you want to add a 'state' key inside 'widgets', do this:
# if 'widgets' in notebook.metadata:
#     notebook.metadata['widgets']['state'] = {}

# Save the modified notebook
with open(output_notebook, 'w') as f:
    nbformat.write(notebook, f)

print(f"Fixed notebook saved as {output_notebook}")


Fixed notebook saved as d_modelos22.ipynb
